# <存放自行包好的，自定義函數區>

In [1]:
import Ipynb_importer  # 支援.ipynb檔案呼叫
import copy # 支援deepcopy函式
#import Main
from itertools import chain

# 自行創建之檔案 (function 或 變數 都可以import)
from data_process import setup_seed, load_dataset, split_data, normalize_data, un_normalize_data, seq_data, batch_data, Univar_SingleStep_seq, Multivariate_SingleStep_seq, Multivariate_MultiStep_seq
from args import Univar_SingleStep_args, Multivariate_SingleStep_args, Multivariate_MultiStep_args
from models import LSTM, Simple_RNN, TCN, TCN_LSTM

#  device還未加入
# tqdm未確認
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

import torch
from scipy.interpolate import make_interp_spline
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# 設置GPU (可放任意.py檔，import device即可呼叫)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

importing Jupyter notebook from data_process.ipynb
importing Jupyter notebook from args.ipynb
importing Jupyter notebook from models.ipynb


## 固定初始化權重

In [2]:
setup_seed(20)

## 運行data_process

In [3]:
def load_data(args, seq_method):  # 
    if seq_method == 'us':
        Train_seq, Val_seq, Test_seq, m, n = Univar_SingleStep_seq(args.seq_len, args.batch_size)  # shape = (batch_size, 2)
    elif seq_method == 'ms':
        Train_seq, Val_seq, Test_seq, m, n = Multivariate_SingleStep_seq(args.seq_len, args.batch_size)
    elif seq_method == 'mm':
        Train_seq, Val_seq, Test_seq, m, n = Multivariate_MultiStep_seq(args.seq_len, args.batch_size, args.output_size, args.output_size)

    return Train_seq, Val_seq, Test_seq, m, n

## 查看模型結構

In [ ]:
def look_model_shape(model):
    print("查看模型結構:\n",model)
    print("\n查看網路參數:")
    for name, parameters in model.named_parameters():
        print(name, ':', parameters.size())
    print("--------------------------------")

## 驗證模型loss

In [4]:
def get_val_loss(model, Val_data):
    model.eval()
    loss_function = nn.MSELoss().to(device)
    val_loss = []
    for (seq, label) in Val_data:
        seq = seq.to(device)
        label = label.to(device)

        with torch.no_grad():  # 不計算梯度
            y_pred = model(seq)
            loss = loss_function(y_pred, label)
            val_loss.append(loss.item())
    return np.mean(val_loss)  # 取所有loss平均值

## 計算模型分數

In [5]:
def get_mape(y, pred):
    """
    y: 正確值
    pred: 預測值
    """
    return np.mean(np.abs((y - pred) / y))

## 畫圖

In [6]:
"""def get_plot2(y, pred):
    x = [i for i in range(1, 151)]
    x_smooth = np.linspace(np.min(x), np.max(x), 900)  # 在 a 到 b 之間產生 900 個點
    y_smooth = make_interp_spline(x, y[150:300])(x_smooth)  # 變成平滑的曲線
    plt.plot(x_smooth, y_smooth, c='green', marker='*', ms=1, alpha=0.75, label='true')

    y_smooth = make_interp_spline(x, pred[150:300])(x_smooth)
    plt.plot(x_smooth, y_smooth, c='red', marker='o', ms=1, alpha=0.75, label='pred')
    plt.grid(axis='y')
    plt.legend()
    plt.show()"""

def get_plot(y, pred):
    # 可視化部分結果
    x = [i for i in range(1, 151)]
    plt.plot(x, y[150:300], c='green', marker='*', ms=1, alpha=0.75, label='true')
    plt.plot(x, pred[150:300], c='red', marker='o', ms=1, alpha=0.75, label='pred')
    #plt.figure(figsize = (8,5))
    #plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=8))  # 設定x軸刻度間距 
    #plt.plot( x , y, label='check_out_gt')
    #plt.plot( x , pred, label='check_out_pred')

    plt.xlabel("date(part interval)")
    plt.ylabel("load")
    #plt.xticks(rotation=30)
    plt.legend()
    plt.show()

## Train

In [7]:
def train(args, Train_data, Val_data, model_path, model_method):
    input_size, hidden_size, num_layers = args.input_size, args.hidden_size, args.num_layers
    output_size = args.output_size
    # MODEL
    if model_method == 'LSTM':
        model = LSTM(input_size, hidden_size, num_layers, output_size, batch_size=args.batch_size).to(device)  # 在GPU跑
    elif model_method == 'TCN':
        #model = TCN(input_size, output_size, [64,128], 2, 0.1).to(device) 
        model = TCN(input_size, output_size, [8,25], 6, 0.1).to(device)  

    # 查看模型結構
    look_model_shape(model)
    # LOSS
    loss_function = nn.MSELoss().to(device)  # MSE loss
    # OPTIMIZER
    if args.optimizer == 'adam':
        optimizer = torch.optim.Adam(params=model.parameters(), lr=args.lr, weight_decay=args.weight_decay)  # 設定optimization
    # LR
    scheduler = StepLR(optimizer, step_size=args.step_size, gamma=args.gamma)
    
    # 訓練開始
    min_epochs = 5  # 最小保存回合
    min_val_loss = 5
    best_model = None
    for epoch in tqdm(range(args.epochs)):
        train_loss = []  # 紀錄loss值
        model.train()
        for (seq, label) in Train_data:
            seq = seq.to(device)  # 在GPU跑
            label = label.to(device)
            
            # y_pred: 預測結果
            y_pred = model(seq)  # 將data傳入model進行forward propagation
            loss = loss_function(y_pred, label)  # 計算loss
            train_loss.append(loss.item())  # 保存loss結果 -> .item()很重要，取值
            
            optimizer.zero_grad()  # 清空前一次的gradient
            loss.backward()  # 根據loss進行back propagation，計算gradient
            optimizer.step()  # 做gradient descent，根據gradient descent更新所有網路參數

            # zero_grad()和backward()使用技巧、Truncated Back Propagation Through Time技術
            ## 參考: https://meetonfriday.com/posts/18392404/
        scheduler.step()
        
        # 驗證
        val_loss = get_val_loss(model, Val_data)  # ~get_val_loss
        if epoch + 1 >= min_epochs and val_loss < min_val_loss:
            min_val_loss = val_loss
            best_model = copy.deepcopy(model)  # 需import copy
        
        
        print('epoch {:03d} train_loss {:.8f} val_loss{:.8f}'.format(epoch, np.mean(train_loss), val_loss))
        
    
    save_model = {'models': best_model.state_dict()}  # 標籤: models
    torch.save(save_model, model_path)

## Test

In [12]:
def test(args, Test_data, m, n, model_path, model_method):  # 需可獨立執行
    pred = []  # 預測值
    y = []  # 正確值
    
    print('Loading Model...')
    input_size, hidden_size, num_layers = args.input_size, args.hidden_size, args.num_layers
    output_size = args.output_size
    # MODEL
    # MODEL
    if model_method == 'LSTM':
        model = LSTM(input_size, hidden_size, num_layers, output_size, batch_size=args.batch_size).to(device)  # 在GPU跑
    elif model_method == 'TCN':
        #model = TCN(input_size, output_size, [64,128], 2, 0.1).to(device) 
        model = TCN(input_size, output_size, [8,25], 6, 0.1).to(device) 
        
    model.load_state_dict(torch.load(model_path)['models'])  # 加載模型參數
    model.eval()
    
    print('Start predicting...')
    for (seq, label) in tqdm(Test_data):
        """
        seq: =final_seq[][0]   -> (x,x,x,x,x,x,x)
        label: =final_seq[][1] -> (y)
        y_pred: 預測結果，shape等同於label(Ground True)
        """
        label = list(chain.from_iterable(label.data.tolist()))  # 嵌套列表 → 合併單一列表
        y.extend(label)
        seq = seq.to(device)  # 進模型資料，用GPU跑
        
        with torch.no_grad():
            y_pred = model(seq)
            #print("before: ",y_pred[0:10])
            #y_pred = y_pred.data.tolist()
            #print("after: ",y_pred[0:10])
            y_pred = list(chain.from_iterable(y_pred.data.tolist()))
            #pred.append(y_pred)  # extend v.s. append: https://sean22492249.medium.com/python-list-的-extend-append-的差別-5dd4a585aafe
            pred.extend(y_pred)
            
    # pred shape = (  , batch_size, 2) # 未確認結構

    # 還原正規化
    y, pred = un_normalize_data(y, pred, m, n)  # ~un_normalize_data
    print("y_shape: ",y.shape)
    print("pred_shape: ",pred.shape)
    print("\ny_value: ",y[0:10])
    print("pred_value: ",pred[0:10],"\n")
    print("mape:", get_mape(y, pred))  # ~get_mape
    
    get_plot(y, pred)  # 顯示分析結果  ~get_plot


# 單獨測試用

In [11]:
# 單獨測試train
if __name__ == '__main__':  #加入這個，被import時，不會執行def以外的code，自己執行才會呼叫
    model_path = './save_models/Univar_SingleStep.pkl'
    args = Univar_SingleStep_args()
    Train_seq, Val_seq, Test_seq, m, n = load_data(args, 'us')  # ~load_data
    train(args, Train_seq, Val_seq, model_path, 'LSTM')  # ~train
    
"""
參考: https://ithelp.ithome.com.tw/articles/10277352
"""

seq_len: 7
總列數: (3000, 8)
序列化_seq_data: (2993, 2)

示範:
seq: tensor([[0.6209],
        [0.5890],
        [0.6001],
        [0.6056],
        [0.6782],
        [0.6538],
        [0.7577]])
label: tensor([0.7906]) 
----------------------------

seq_len: 7
總列數: (1000, 8)
序列化_seq_data: (993, 2)

示範:
seq: tensor([[0.4345],
        [0.4440],
        [0.4058],
        [0.4105],
        [0.3603],
        [0.3877],
        [0.5956]])
label: tensor([0.6223]) 
----------------------------

seq_len: 7
總列數: (1000, 8)
序列化_seq_data: (993, 2)

示範:
seq: tensor([[0.6854],
        [0.6605],
        [0.6447],
        [0.6572],
        [0.6823],
        [0.5961],
        [0.4972]])
label: tensor([0.4261]) 
----------------------------

epoch 000 train_loss 0.29345316 val_loss0.46423262
epoch 001 train_loss 0.29345316 val_loss0.46423262


KeyboardInterrupt: 

In [ ]:
# 單獨測試test
if __name__ == '__main__':
    model_path = './save_models/Univar_SingleStep.pkl'
    args = Univar_SingleStep_args() 
    Train_seq, Val_seq, Test_seq, m, n = load_data(args, 'us')  # ~load_data
    test(args, Test_seq, m, n, model_path, 'LSTM')  # ~test